In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import sys
import os
import copy
import pathlib
import time

import numpy as np
import matplotlib.pyplot as plt
import scipy

import torch
import torchvision
import torchvision.transforms as transforms

# for creating validation set
from sklearn.model_selection import train_test_split

%matplotlib notebook
%matplotlib notebook
# %matplotlib inline
# %matplotlib qt

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!git clone "https://github.com/RichieHakim/basic_neural_processing_modules.git" "/content/drive/MyDrive/00 - ROI/basic_neural_processing_modules"
!cd "/content/drive/MyDrive/00 - ROI/basic_neural_processing_modules" && git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path '/content/drive/MyDrive/00 - ROI/basic_neural_processing_modules' already exists and is not an empty directory.
Already up to date.


In [4]:
!pip install pycuda

In [5]:
import sys

# sys.path.append(r'/media/rich/Home_Linux_partition/github_repos')
# sys.path.append('/Users/Josh/Documents/github_repos/')
sys.path.append(r'/content/drive/MyDrive/00 - ROI')
dir_folders = r'/content/drive/MyDrive/00 - ROI/label_data'


%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import torch_helpers, decomposition, math_functions, classification
from GCaMP_ROI_classifier.new_stuff import util, models, training_simCLR, augmentation, training_classHead, training_supervised

## Import training data

In [6]:
# import pickle

# with open('/content/drive/MyDrive/00 - ROI/label_data/all_stat_files_20211022.pkl', 'rb') as file:
# # with open('/Users/Josh/Documents/Harvard/label_data/all_stat_files_20211022.pkl', 'rb') as file:
# # with open('/media/rich/bigSSD/for_Josh/GCAMP_ROI_Classifier_data/label_data/all_stat_files_20211022.pkl', 'rb') as file:
#     statFiles_scraped = pickle.load(file)

In [7]:
# keys_toDelete = [
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\for Loic\\16x_analysis20200514\\Suite2p and MC for denoised movie\\suite2p\\plane0\\stat.npy',
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\round 4 experiments\\mouse 6.28\\20201102\\suite2p_accidentally_run_on_everything\\plane0\\stat.npy',
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\for Loic\\20x\\mouse 2.5\\20200306\\suite2p attemp 2 _tau 1_5\\plane0\\stat.npy',
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\for Loic\\20x\\mouse 2.5\\20200306\\suite2p attempt 1 _tau 2\\plane0\\stat.npy',
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\for Loic\\20x\\mouse 2.5\\20200306\\suite2p attempt 3 _tau 1\\plane0\\stat.npy',
#     '\\\\research.files.med.harvard.edu\\Neurobio\\MICROSCOPE\\Rich\\data\\res2p\\scanimage data\\for Loic\\AAV\\oldBMIrampExp_20191112_mouse1013A\\ROI extraction\\suite2p\\plane0\\stat.npy',
# ]
# for key in keys_toDelete:
#     del statFiles_scraped[key]

In [8]:
# # keys_toDelete = list(statFiles_scraped.keys())[:len(statFiles_scraped.keys())]
# keys_toDelete = list(statFiles_scraped.keys())[2:]
# for key in keys_toDelete:
#     del statFiles_scraped[key]

In [9]:
# sf_list = util.convert_multiple_stat_files(statFiles_dict=statFiles_scraped, out_height_width=[36,36], max_footprint_width=901, plot_pref=False)

# # images_all = np.concatenate(sf_list, axis=0)
# images_all = np.concatenate([sf_list[ii][:1000] for ii in range(len(sf_list))], axis=0)

# images_all.shape

In [10]:
# images_all_norm = (images_all / np.max(images_all, axis=(1,2), keepdims=True)) * 1

In [11]:
# # create validation set
# # X_train, X_val, y_train, y_val = train_test_split(images[:], labels[:], test_size = 0.15)
# X_train, X_val, y_train, y_val = train_test_split(images_all_norm, np.arange(images_all.shape[0]), test_size = 0.15)
# (X_train.shape, y_train.shape), (X_val.shape, y_val.shape)

In [12]:
dir_folders = r'/content/drive/MyDrive/00 - ROI/label_data'
# dir_folders = r'/media/rich/bigSSD/for_Josh/GCAMP_ROI_Classifier_data/label_data'
# dir_folders = r'/users/Josh/Documents/Harvard/label_data'
folders = [r'mouse 6_28 _ day 20200903/',
             r'mouse6_28 _ day20200815/']
fileNames_statFiles = [r'stat.npy']*len(folders)
paths_statFiles = [pathlib.Path(dir_folders) / folders[ii] / fileNames_statFiles[ii] for ii in range(len(folders))]

sf_all = util.import_multiple_stat_files(   paths_statFiles=paths_statFiles,
                                            fileNames_statFiles=fileNames_statFiles,
                                            out_height_width=[32*3,32*3],
                                            max_footprint_width=241,
                                            plot_pref=True)
images_labeled = np.concatenate(sf_all, axis=0)
images_labeled = (images_labeled / np.max(images_labeled, axis=(1,2), keepdims=True)) * 1
print(f'concatenated images shape: {images_labeled.shape}')

fileNames_labelFiles = ['labels_posthoc_filledIn_allCells.npy',
             'labels_posthoc_all.npy']
paths_labelFiles = [pathlib.Path(dir_folders) / folders[ii] / fileNames_labelFiles[ii] for ii in range(len(folders))]

labels_all = util.import_multiple_label_files(paths_labelFiles=paths_labelFiles,
                                       plot_pref=True)
labels = np.concatenate(labels_all)

assert np.alltrue([sf_all[ii].shape[0] == labels_all[ii].shape[0] for ii in range(len(sf_all))]) , 'num images in stat files does not correspond to num labels'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

concatenated images shape: (9715, 96, 96)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# tmpd = pd.DataFrame(labels)
# tmpd[1] = tmpd[0]
# tmpd.groupby(0).count()

In [14]:
#code doesn't work with multiple open number maybe (?)

In [15]:
labels = classification.squeeze_integers(labels)
images_labeled = images_labeled[labels != 2]
labels = labels[labels != 2]
labels = classification.squeeze_integers(labels)

In [16]:
# images_labeled = images_labeled[labels!=3]
# labels = labels[labels!=3]


## Balance classes of inputs

In [17]:
duplicates = 1
balanced = False

images_dup = np.tile(images_labeled , (duplicates , 1 , 1))
labels_dup = np.tile(labels , (duplicates))

print(images_dup.shape)
print(labels_dup.shape)

if balanced:
  numToGetTo = np.sum(labels_dup==0)
  print(numToGetTo)
  for ii in np.array([1,4,5]):
  #     idxToDelete = np.cumsum(labels_dup==ii) <= (np.sum(labels_dup==ii) - numToGetTo)
      if ii==5:
          numToGetTo = np.sum(labels_dup==0)/2
      else:
          numToGetTo = np.sum(labels_dup==0)

      idxToDelete = (np.cumsum(labels_dup==ii) * (labels_dup==ii)) > numToGetTo
      images_dup = images_dup[idxToDelete==0,:,:]
      labels_dup = labels_dup[idxToDelete==0]





print(images_dup.shape)
print(labels_dup.shape)

numToGetTo = np.sum(labels_dup==0)
print(numToGetTo)

print(np.sum(labels_dup==0))
print(np.sum(labels_dup==1))
print(np.sum(labels_dup==4))
print(np.sum(labels_dup==5))
print(images_dup.shape)
print(labels_dup.shape)

(9714, 96, 96)
(9714,)
(9714, 96, 96)
(9714,)
532
532
620
0
0
(9714, 96, 96)
(9714,)


In [18]:
# create validation set

###### REMOVE WITH ENOUGH RAM
images = images_dup
labels = labels_dup

# X_train, X_val, y_train, y_val = train_test_split(images[:], labels[:], test_size = 0.15)
X_labeled_train, X_labeled_val, y_labeled_train, y_labeled_val = train_test_split(images_dup, labels_dup, test_size = 0.15)
X_train, X_val, y_train, y_val = X_labeled_train, X_labeled_val, y_labeled_train, y_labeled_val
(X_labeled_train.shape, y_labeled_train.shape), (X_labeled_val.shape, y_labeled_val.shape)

(((8256, 96, 96), (8256,)), ((1458, 96, 96), (1458,)))

In [19]:
# # create validation set
# # X_train, X_val, y_train, y_val = train_test_split(images[:], labels[:], test_size = 0.15)
# X_labeled_train, X_labeled_val, y_labeled_train, y_labeled_val = train_test_split(images_dup[labels!=3], labels_dup[labels!=3], test_size = 0.15)
# (X_labeled_train.shape, y_labeled_train.shape), (X_labeled_val.shape, y_labeled_val.shape)

In [20]:
plt.figure()
plt.hist(labels_dup, 20);

<IPython.core.display.Javascript object>

## Define augmentation pipeline

In [21]:
X_train.shape

(8256, 96, 96)

In [22]:
import torchvision.transforms    

transforms = torch.nn.Sequential(
    
# ######     torchvision.transforms.RandomAdjustSharpness(torch.rand(1)*5, p=0.5),
    
#     torchvision.transforms.RandomHorizontalFlip(p=0.5),
        
#     torchvision.transforms.GaussianBlur(kernel_size=5,
#                                         sigma=(0.0001, 0.5)),
    
#     torchvision.transforms.RandomPerspective(distortion_scale=0.3, 
#                                              p=0.5, 
#                                              interpolation=torchvision.transforms.InterpolationMode.BILINEAR, 
#                                              fill=0),
#     torchvision.transforms.RandomAffine(
#                                         degrees=(-180,180),
#                                         translate=(0.1, 0.1),
#                                         scale=(3*0.7, 3*1.7), 
#                                         shear=(-20, 20, -20, 20), 
#                                         interpolation=torchvision.transforms.InterpolationMode.BILINEAR, 
#                                         fill=0, 
#                                         fillcolor=None, 
#                                         resample=None),
    
#     augmentation.AddPoissonNoise(   scaler_bounds=(10, 50),
#                                     prob=1,
#                                     base=100,
#                                     scaling='log'),

#     augmentation.AddGaussianNoise(  mean=0, 
#                                     std=0.07,
#                                     prob=1),
)
scripted_transforms = torch.jit.script(transforms)
# scripted_transforms = transforms



# dataset_train = util.dataset_simCLR(
#                                     X_train, 
#                                     y_train, 
# #                                     torch.tensor(X_train), 
# #                                     y_train, 
#                                     n_transforms=2, 
# #                                     class_weights=[1]*4,
# #                                     class_weights=[0.34, 0.33, 0.33, 0.00],
# #                                     class_weights=[0.0, 0.0, 0.0, 1.00],
# #                                     class_weights=np.array([0.34, 0.33, 0.33, 0.00])[y_train],
# #                                     class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
#                                     class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
#                                     transform=scripted_transforms,
#                                     DEVICE='cpu',
#                                     dtype_X=torch.float32,
#                                     dtype_y=torch.int64 )

# dataloader_train = torch.utils.data.DataLoader( dataset_train,
#                                                 batch_size=2048,
#             #                                     sampler=sampler,
#                                                 shuffle=True,
#                                                 drop_last=True,
#                                                 pin_memory=True,
#                                                 num_workers=0,
# #                                                 num_workers=32,
# #                                                 prefetch_factor=0,
# #                                                 persistent_workers=True,
#                                                 )

# # dataset_val = util.dataset_simCLR(torch.tensor(X_val), 
# #                                     y_val, 
# #                                     n_transforms=2, 
# #                                     transform=scripted_transforms,
# #                                     DEVICE='cpu',
# #                                     dtype_X=torch.float32,
# #                                     dtype_y=torch.int64 )

# # dataloader_val = torch.utils.data.DataLoader(   dataset_val,
# #                                                 batch_size=128,
# #             #                                     sampler=sampler,
# #                                                 shuffle=True,
# #                                                 drop_last=True,
# #                                                 pin_memory=True,
# #                                                 num_workers=0,
# #                                                 )         



In [23]:

# dataset_simCLR = torch.DataSet()

In [24]:



# dataset_train = util.dataset_simCLR(
#                                     X_train, 
#                                     y_train, 
# #                                     torch.tensor(X_train), 
# #                                     y_train, 
#                                     n_transforms=2, 
# #                                     class_weights=[1]*4,
# #                                     class_weights=[0.34, 0.33, 0.33, 0.00],
# #                                     class_weights=[0.0, 0.0, 0.0, 1.00],
# #                                     class_weights=np.array([0.34, 0.33, 0.33, 0.00])[y_train],
# #                                     class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
#                                     class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
#                                     transform=scripted_transforms,
#                                     DEVICE='cpu',
#                                     dtype_X=torch.float32,
#                                     dtype_y=torch.int64 )

# dataloader_train = torch.utils.data.DataLoader( dataset_train,
#                                                 batch_size=2048,
#             #                                     sampler=sampler,
#                                                 shuffle=True,
#                                                 drop_last=True,
#                                                 pin_memory=True,
#                                                 num_workers=0,
# #                                                 num_workers=32,
# #                                                 prefetch_factor=0,
# #                                                 persistent_workers=True,
#                                                 )


In [25]:
# import time

# %matplotlib inline
# tic=time.time()
# for idx in range(5):
# # for ii,idx in enumerate(np.random.randint(0,X_train.shape[0], 5)):
# #     for jj in range(im[0])
# #     print(time.time() - tic)
# #     tic = time.time()
# #     print((im[0][0]).shape)
#     print(dataset_train[idx][3])
#     fig, axs = plt.subplots(2)
#     for jj, ax in enumerate(axs):
# #         ax.imshow(im[0][j].cpu().squeeze())
# #         ax.imshow(dataset_train[idx][0][jj][0].permute(1,2,0).cpu().squeeze()/20)
#         im_toShow = dataset_train[idx][0][jj][0].cpu().squeeze()
#         ax.imshow(im_toShow)
# #     if ii > 5:
# #         break

## Train

In [26]:
### REMOVE

DEVICE = torch_helpers.set_device(use_GPU=True)
# DEVICE = torch_helpers.set_device(use_GPU=False)


device: 'cuda'


In [27]:
# import torchvision.models

# model = torchvision.models.resnet18(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False


# # for param in model.layer3.parameters():
# #     param.requires_grad = True
# for param in model.layer4.parameters():
#     param.requires_grad = True
# for param in model.avgpool.parameters():
#     param.requires_grad = True
# for param in model.fc.parameters():
#     param.requires_grad = True

In [28]:
# import torchvision.models

# model = torchvision.models.resnet101(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False

In [29]:
class ModelTackOn(torch.nn.Module):
  def __init__(self, base_model, pre_head_fc_sizes=[100], post_head_fc_sizes=[100], classifier_fc_sizes=None):
    super(ModelTackOn, self).__init__()
    self.base_model = base_model
    final_base_layer = list(base_model.children())[-1]
    # final_base_layer = list(list(model.children())[-1].children())[-1]
    # print(final_base_layer)

    self.pre_head_fc_lst = []
    self.post_head_fc_lst = []
    self.classifier_fc_lst = []

    for i, pre_head_fc in enumerate(pre_head_fc_sizes):
      if i == 0:
        in_features = final_base_layer.out_features if hasattr(final_base_layer,'out_features') else 512
      else:
        in_features = pre_head_fc_sizes[i - 1]
      fc_layer = torch.nn.Linear(in_features=in_features, out_features=pre_head_fc)
      self.add_module(f'PreHead_{i}', fc_layer)
      self.pre_head_fc_lst.append(fc_layer)

    
    for i, post_head_fc in enumerate(post_head_fc_sizes):
      if i == 0:
        in_features = pre_head_fc_sizes[-1]
      else:
        in_features = post_head_fc_sizes[i - 1]
      fc_layer = torch.nn.Linear(in_features=in_features, out_features=post_head_fc)
      self.add_module(f'PostHead_{i}', fc_layer)
      self.post_head_fc_lst.append(fc_layer)
    
    if classifier_fc_sizes is not None:
      for i, classifier_fc in enumerate(classifier_fc_sizes):
        if i == 0:
          in_features = pre_head_fc_sizes[-1]
        else:
          in_features = classifier_fc_sizes[i - 1]
        fc_layer = torch.nn.Linear(in_features=in_features, out_features=classifier_fc)
        self.add_module(f'Classifier_{i}', fc_layer)
        self.classifier_fc_lst.append(fc_layer)
    
  def forward(self, X):
    interim = self.base_model(X)
    interim = self.get_head(interim)
    interim = self.get_latent(interim)
    return interim

  def get_head(self, base_out):
    # print('base_out', base_out.shape)
    head = base_out
    for pre_head_layer in self.pre_head_fc_lst:
      # print('pre_head_layer', pre_head_layer.in_features)
      head = pre_head_layer(head)
      # print('head', head.shape)
    return head
  
  def get_latent(self, head):
    latent = head
    for post_head_layer in self.post_head_fc_lst:
      latent = post_head_layer(latent)
    return latent
  
  def classify(self, head):
    logit = head
    for classifier_layer in self.classifier_fc_lst:
      logit = classifier_layer(logit)
    return logit

  def set_pre_head_grad(self, requires_grad=True):
    for pre_head_layer in self.pre_head_fc_lst:
      pre_head_layer.requies_grad = requires_grad
  
  def set_post_head_grad(self, requires_grad=True):
    for post_head_layer in self.post_head_fc_lst:
      post_head_layer.requies_grad = requires_grad
    
  def set_classifier_grad(self, requires_grad=True):
    for classifier_layer in self.classifier_fc_lst:
      classifier_layer.requies_grad = requires_grad

  def prep_contrast(self):
    self.set_pre_head_grad(requires_grad=True)
    self.set_post_head_grad(requires_grad=True)
    self.set_classifier_grad(requires_grad=False)

  def prep_classifier(self):
    self.set_pre_head_grad(requires_grad=False)
    self.set_post_head_grad(requires_grad=False)
    self.set_classifier_grad(requires_grad=True)
  

In [30]:
import torchvision.models

# base_model = torchvision.models.resnet101(pretrained=True)
base_model = torchvision.models.resnet18(pretrained=True)





for param in base_model.parameters():
    param.requires_grad = False

retrain = list(base_model.children())[-1:]
for layer in retrain:
  params = layer.parameters()
  for param in params:
    param.requires_grad = True

In [31]:
import torchvision.models

# base_model_frozen = torchvision.models.resnet101(pretrained=True)
base_model_frozen = torchvision.models.resnet18(pretrained=True)
for param in base_model_frozen.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
# DEVICE='cpu'

# logreg = LogisticRegression(solver='liblinear')
# base_model = base_model.to(DEVICE)
# features_train = base_model(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)).detach().cpu()
# logreg.fit(features_train, y_labeled_train)
# print(logreg.score(features_train, y_labeled_train))

# features_val = base_model(util.tile_channels(torch.as_tensor(X_labeled_val[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)).detach().cpu()
# y_hat = logreg.predict_proba(features_val)

# cm = classification.confusion_matrix(y_hat, y_labeled_val)
# cm

In [34]:
# plt.imshow(cm)
# plt.colorbar()

In [35]:
# logreg.score(features_val, y_labeled_val)

In [36]:
# model

In [37]:
sq = torch.nn.Sequential(*(list(base_model_frozen.children())[:-1] + [torch.nn.Flatten()]))


In [38]:
base_model_frozen

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
model = ModelTackOn(sq.to(DEVICE), pre_head_fc_sizes=[len(np.unique(y_train))], post_head_fc_sizes=[])

# model = torch.nn.Sequential([sq.to(DEVICE), torch.nn.Linear], pre_head_fc_sizes=[len(np.unique(y_train))], post_head_fc_sizes=[])


# model = ModelTackOn(base_model_frozen, pre_head_fc_sizes=[len(np.unique(y_train))], post_head_fc_sizes=[])
# model = ModelTackOn(base_model, pre_head_fc_sizes=[len(np.unique(y_train))], post_head_fc_sizes=[])
model.train()

ModelTackOn(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv

In [40]:
base_model_frozen

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [41]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


# model = models.LeNet1(dropout_prob=0.3, momentum_val=0, n_output_features=64)


criterion = [CrossEntropyLoss(), torch.nn.MSELoss()]
# criterion = CrossEntropyLoss(reduction='none')

# optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=0.0)
optimizer = Adam(model.parameters(), lr=2e-2)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,
                                                   gamma=1-0.0002,
#                                                    gamma=1,
                                                  )
model.to(DEVICE)
criterion = [_.to(DEVICE) for _ in criterion]
losses_train, losses_val, val_accs, acc = [], [np.nan], [], []

In [42]:
from torch.utils.data import Dataset


In [43]:
pct_batch_size = 1.0

dataset_train = util.dataset_supervised(
                                    X_train,
                                    y_train,
                                    class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
                                    transform=scripted_transforms,
                                    DEVICE='cpu',
                                    dtype_X=torch.float32,
                                    dtype_y=torch.int64
                                    )

dataloader_train = torch.utils.data.DataLoader( dataset_train,
                                                batch_size=int(pct_batch_size*X_labeled_train.shape[0]),
                                                # batch_size=64,
                                                shuffle=True,
                                                drop_last=True,
                                                pin_memory=True,
                                                # num_workers=16,
                                                # persistent_workers=True,
                                                # prefetch_factor=0
                                                )



In [44]:
# model(dataset_train[0])

In [45]:

import sys
sys.getsizeof(X_labeled_train)/1e9

0.608698496

In [46]:
# for dl in dataloader_train:
#   print(dl[0].shape)
#   break

In [47]:
from sklearn.metrics import confusion_matrix

def get_cm(pred_cm, y_cm, plot=False):
  cm = confusion_matrix(y_cm, np.argmax(pred_cm, -1))
  cm = cm / cm.sum(0, keepdims=True)
  
  # cm = classification.confusion_matrix(y_hat, y_labeled_val)
  # print(cm)
  
  if plot:
    plt.figure()
    plt.imshow(cm)
    plt.colorbar()
    plt.show()
  
  return cm

In [48]:
x_feed_through_tr = util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)
x_feed_through_val = util.tile_channels(torch.as_tensor(X_labeled_val[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)

cpu_tr = sq(x_feed_through_tr)
cpu_val = sq(x_feed_through_val)
cpu_tr.shape, cpu_val.shape, cpu_tr.squeeze().shape, cpu_val.squeeze().shape

RuntimeError: ignored

In [ ]:
num_tr_ex = X_labeled_val.shape[0]

# logreg = LogisticRegression(solver='liblinear')
# logreg = LogisticRegression(solver='liblinear', penalty='none', )
# logreg = LogisticRegression(solver='newton-cg', penalty='none', max_iter=4000)
logreg = LogisticRegression(solver='lbfgs')
# logreg = LogisticRegression(solver='lbfgs', penalty='none', max_iter=4000)

# base_features_train = base_model_frozen(x_feed_through_tr).detach().cpu()
base_features_train = cpu_tr.cpu().detach().numpy()
logreg.fit(base_features_train, y_labeled_train)

# base_features_val = base_model_frozen(x_feed_through_val).detach().cpu()
base_features_val = cpu_val.cpu().detach().numpy()

# base_model_frozen.to('cpu')
# X_labeled_train.to('cpu')

logistic_pred_train = get_cm(logreg.predict_proba(base_features_train), y_labeled_train)
logistic_pred_val = get_cm(logreg.predict_proba(base_features_val), y_labeled_val)


x_feed_through_tr.to(DEVICE)
x_feed_through_val.to(DEVICE)

In [ ]:
x_feed_through_tr.shape, x_feed_through_val.shape

In [ ]:
logistic_pred_train

In [ ]:
logistic_pred_val

In [ ]:
# prm = []

# for param in model.parameters():
#   if param.requires_grad:
#     print(param.shape)
#     prm.append(param)
# (logreg.coef_ - prm[0].cpu().detach().numpy()), (logreg.intercept_ - prm[1].cpu().detach().numpy())

In [ ]:
for i in range(1):
  torch_helpers.show_all_tensors(globals())

In [ ]:
# torch_helpers.delete_all_cuda_tensors(globals())

import gc

torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# figure out what % of the dataset is being used
torch_helpers.show_all_tensors(globals())


In [ ]:
x_feed_through_tr.to(DEVICE)
x_feed_through_val.to(DEVICE)
cpu_tr.to(DEVICE)
cpu_val.to(DEVICE)
model.to(DEVICE)
base_model_frozen.to(DEVICE)



penalized_params = list(model.modules())[-1].parameters()
penalized_params = torch.cat([_.view(-1) for _ in penalized_params], -1)

l2_alpha = 0.1

n_epochs=300000
for epoch in range(n_epochs):
    print(f'epoch: {epoch}')
    training_supervised.epoch_step(dataloader_train, 
                                    model, 
                                    optimizer, 
                                    criterion, 

                                    penalized_params, l2_alpha,

                                    scheduler=scheduler,
                                    loss_rolling_train=losses_train, 
                                    device=DEVICE, 
                                    do_validation=False,
                                    loss_rolling_val=losses_val,
                                    verbose=2,
                                    verbose_update_period=100
                                   )
    
    torch_helpers.show_all_tensors(globals())
    
    features_train = model(x_feed_through_tr)
    features_train = features_train.cpu().detach().numpy()
    features_val = model(x_feed_through_val)
    features_val = features_val.cpu().detach().numpy()
    # y_hat = scipy.special.softmax(features_val, axis=-1) # logreg.predict_proba(features_val)
    
    print('Training Confusion Matrix')
    print(get_cm(features_train, y_labeled_train))
    print()
    print(logistic_pred_train)

    print()
    print()
    print()

    print('Val Confusion Matrix')
    print(get_cm(features_val, y_labeled_val))
    print()
    print(logistic_pred_val)

    # model.to(DEVICE)


In [ ]:
break

In [ ]:
torch.concatenate

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,10))
plt.plot(losses_train, label='Training Loss')
plt.plot(losses_val, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:

# del thing

In [ ]:
# import gc

# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


# model = models.LeNet1(dropout_prob=0.3, momentum_val=0, n_output_features=64)


# criterion = CrossEntropyLoss()
criterion = CrossEntropyLoss(reduction='none')

optimizer = Adam(model.parameters(), lr=2e-3, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,
                                                   gamma=1-0.0002,
#                                                    gamma=1,
                                                  )
# val_obj = training_simCLR.validation_Obj(   X_val,
#                                             y_val,
#                                             model,
#                                             criterion,
#                                             DEVICE=DEVICE,
#                                             dtype_X=torch.float32,
#                                             dtype_y=torch.int64) # Needs to take in weights

model.to(DEVICE)
criterion = criterion.to(DEVICE)

# # classHead = HeadModel(model, RandomForestClassifier)
# classHead = training_classHead.HeadModel(model, LogisticRegression)
# dataset_train.headmodel = classHead
# # dataset_train.headmodel = None

losses_train, losses_val, val_accs, acc = [], [np.nan], [], []

In [ ]:
import pandas as pd
tracking_df_cols = ['sample_id',
                    'epoch',
                    'batch_iter',
                    'labels'
                    ]
tracking_df = pd.DataFrame(columns=tracking_df_cols)

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
%matplotlib notebook

logreg = LogisticRegression(solver='liblinear', max_iter=30)
# logreg_predict_head = LogisticRegression(solver='liblinear')
dataset_train.classification_model = None



# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,
#                                                    gamma=1-0.001,
# #                                                    gamma=1,
#                                                   )
n_epochs=300000
for epoch in range(n_epochs):
    print(f'epoch: {epoch}')

    model.prep_contrast()
    training_simCLR.epoch_step( dataloader_train, 
                                model, 
                                optimizer, 
                                criterion,
                                scheduler=scheduler, 
                                temperature=0.5,
                                loss_rolling_train=losses_train, 
                                device=DEVICE, 
                                do_validation=False,
#                                 validation_Object=val_obj,
                                loss_rolling_val=losses_val,
                                verbose=2,
                                verbose_update_period=100,
                               )
    

    model.prep_classifier()

    # print(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1).shape)

    features_train = model.get_head(model.base_model(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1))).detach().cpu()
    # features_train = model(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)).detach().cpu()
    # features_train = model(torch.as_tensor(X_labeled_train, device=DEVICE, dtype=torch.float32)[:,None,...]).detach().cpu()
    # features = model(torch.tensor(X_train[y_train != 3], device=DEVICE, dtype=torch.float32)[:,None,...]).detach().cpu()
    
    tic = time.time()
    logreg.fit(features_train, y_labeled_train)
    print(time.time() - tic)
    acc.append(logreg.score(features_train, y_labeled_train))
    print(f'acc: {acc[-1]}')
    
    dataset_train.net_model = copy.deepcopy(model).to('cpu')
    dataset_train.classification_model = logreg
    

#     sample_id_num = np.arange(X_labeled_val.shape[0])
#     epoch_val = epoch
#     batch_val = -1
#     p_tmp = logreg.predict_proba(model(torch.as_tensor(util.tile_channels(X_labeled_val), device=DEVICE, dtype=torch.float32)).detach().cpu())
#     logits = p_tmp
#     # logits = np.log(1/(1/p_tmp - 1))

#     col_vals = [sample_id_num, epoch_val, batch_val, y_labeled_val]
#     setup = np.empty((len(sample_id_num), len(col_vals)))
#     for icv, col_val in enumerate(col_vals):
#       setup[:, icv] = col_val
#     tmp_tracking_np = np.concatenate([setup, logits], axis=1)

#     tmp_tracking_df = pd.DataFrame(tmp_tracking_np, index=sample_id_num, columns=tracking_df_cols + [f'logits_{i}' for i in range(logits.shape[1])])
#     tracking_df = tracking_df.append(tmp_tracking_df, ignore_index=True)
#     display(tracking_df)


    
    features_val = model.get_head(model.base_model(util.tile_channels(torch.as_tensor(X_labeled_val[:,None,...], device=DEVICE, dtype=torch.float32), dim=1))).detach().cpu()


    # logreg_predict_head.fit(features_train, y_labeled_train)
    # y_hat = logreg_predict_head.predict_proba(features_val)

    y_hat = logreg.predict_proba(features_val)
    
    cm = classification.confusion_matrix(y_hat, y_labeled_val)
#     plt.figure()
#     plt.imshow(cm)
#     plt.colorbar()
#     plt.show()
    print(cm)


    # tracking_df = tracking_df.append(pd.DataFrame([np.array([100, 0, 0, 0])], index=tracking_df_cols), ignore_index=True)
    
    # model predict
    # Update model in DS
    # get item calls model for each sample
    # output
    # X sample weights predictions
    
#     classHead.fit(X_train[:, None, :, :], y_train, solver='liblinear')
    
#     proba = classHead.predict_proba(X_train[:, None, :, :])
#     class_weights = proba.sum(axis=0)
#     total_num = class_weights.sum()
    
#     eps = 1e-4
    
#     class_weights[class_weights <= 3] = total_num
#     weightings = class_weights.sum()/class_weights
#     final_weights = weightings / weightings.sum()
#     final_weights = np.array([1/proba.shape[1] for _ in range(proba.shape[1])])
    
#     print(class_weights)

#     dataset_train.set_classweights(final_weights)
    
#     print('dataset_train.final_weights', dataset_train.class_weights)


In [ ]:
# # !cd "/content/drive/MyDrive/00 - ROI/GCaMP_ROI_classifier" && git add old_stuff
# # !cd "/content/drive/MyDrive/00 - ROI/GCaMP_ROI_classifier" && git status
# # !cd "/content/drive/MyDrive/00 - ROI/GCaMP_ROI_classifier" && git commit -m "Introducing ResNet and validating process"
# !cd "/content/drive/MyDrive/00 - ROI/GCaMP_ROI_classifier" && git push

In [ ]:
# !git config --global user.email "$indianajonesjz@gmail.com"
# !git config --global user.name "$joshuazimmer"

In [ ]:
plt.figure()
plt.plot(losses_train)

In [ ]:
plt.figure()
plt.plot(acc)

In [ ]:
# DEVICE = 'cpu'

In [ ]:
from sklearn.linear_model import LogisticRegression

model.to(DEVICE)

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
del features_train

In [ ]:
logreg = LogisticRegression(solver='liblinear')
features_train = model(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)).detach().cpu()
# features = model(torch.tensor(X_train[y_train != 3], device=DEVICE, dtype=torch.float32)[:,None,...]).detach().cpu()
logreg.fit(features_train, y_labeled_train)

In [ ]:
y_hat = logreg.predict_proba(features_train)

cm = classification.confusion_matrix(y_hat, y_labeled_train)

print(cm)

plt.figure()
plt.imshow(cm)
plt.colorbar()

In [ ]:
logreg = LogisticRegression(solver='liblinear')
features_train = model(util.tile_channels(torch.as_tensor(X_labeled_train[:,None,...], device=DEVICE, dtype=torch.float32), dim=1)).detach().cpu()
# features = model(torch.tensor(X_train[y_train != 3], device=DEVICE, dtype=torch.float32)[:,None,...]).detach().cpu()
logreg.fit(features_train, y_labeled_train)
y_hat = logreg.predict_proba(features_train)

cm = classification.confusion_matrix(y_hat, y_labeled_train)

print(cm)

plt.figure()
plt.imshow(cm)
plt.colorbar()

In [ ]:
logreg.fit(features_embedded[:,:200], y_labeled_train)
y_hat = logreg.predict_proba(features_embedded[:,:200])

cm = classification.confusion_matrix(y_hat, y_labeled_train)

print(cm)

plt.figure()
plt.imshow(cm)
plt.colorbar()

## Check embeddings

In [ ]:
torch.cuda.empty_cache()

In [ ]:
### REMOVE

DEVICE='cuda'
# DEVICE='cpu'

In [ ]:
model.to(DEVICE)

In [ ]:
features = model(util.tile_channels(torch.tensor(images_labeled, device=DEVICE, dtype=torch.float32)[:,None,...], dim=1)).detach()
# features = model(torch.tensor(images_labeled[labels != 3], device=DEVICE, dtype=torch.float32)[:,None,...]).detach()
# features = model(torch.tensor(X_val, device=DEVICE, dtype=torch.float32)[:,None,...]).detach()
# features = model(torch.tensor(X_train, device=DEVICE, dtype=torch.float32)[:,None,...]).detach()

In [ ]:
_, features_embedded, _, evr = decomposition.torch_pca(features, device=DEVICE, return_cpu=True)

In [ ]:
from sklearn import manifold

tsne = manifold.TSNE(n_components=2, 
                     perplexity=80.0, 
                     early_exaggeration=12.0, 
                     learning_rate=200, 
                     n_iter=1000, 
                     n_iter_without_progress=300, 
                     min_grad_norm=1e-07, 
                     metric='euclidean', 
                     init='pca', 
                     verbose=0, 
                     random_state=None, 
                     method='barnes_hut', 
                     angle=0.5, 
                     n_jobs=-1, 
#                      square_distances='legacy'
                    )
# features_embedded = tsne.fit_transform(features.cpu())
features_embedded = tsne.fit_transform(features_embedded[:,:5].cpu())

In [ ]:
import matplotlib as mpl

# mpl.rcParams['image.cmap'] = 'Set1'
%matplotlib notebook
plt.figure()
plt.scatter(features_embedded[:,0], features_embedded[:,1], c=labels)
# plt.scatter(features_embedded[:,0], features_embedded[:,1], c=labels[labels!=3])
# plt.scatter(features_embedded[:,0], features_embedded[:,1], c=y_val)
# plt.scatter(features_embedded[:,4], features_embedded[:,5], c=y_train)
# plt.scatter(features_embedded[:,11], features[:,43].cpu(), c=y_train)
mpl.rcParams['image.cmap'] = 'viridis'


In [ ]:
plt.figure()
plt.imshow(features.cpu().detach(), aspect='auto', interpolation='none')

In [ ]:
plt.figure()
plt.plot(evr)
plt.yscale('log')

## Check filters

In [ ]:
list(model.state_dict().keys())

In [ ]:
layer_1 = model.state_dict()['cnn_layers.0.weight'].cpu()
layer_2 = model.state_dict()['cnn_layers.2.weight'].cpu()
layer_3 = model.state_dict()['cnn_layers.5.weight'].cpu()
layer_4 = model.state_dict()['cnn_layers.8.weight'].cpu()

plt.figure()
for ii in range(layer_1.shape[1]):
    for jj in range(layer_1.shape[0]):
        plt.subplot2grid((layer_1.shape[1],layer_1.shape[0]),(ii,jj))
        fig = plt.imshow(layer_1[jj,ii,:,:] , clim=(-0.2,0.2))
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

plt.figure()
for ii in range(layer_2.shape[1]):
    for jj in range(layer_2.shape[0]):
        plt.subplot2grid((layer_2.shape[1],layer_2.shape[0]),(ii,jj))
        fig = plt.imshow(layer_2[jj,ii,:,:], clim=(-.05,.05))
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

plt.figure()
for ii in range(16):
    for jj in range(16):
        plt.subplot2grid((16,16),(ii,jj))
        fig = plt.imshow(layer_3[jj, ii,:,:])
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        
plt.figure()
for ii in range(16):
    for jj in range(16):
        plt.subplot2grid((16,16),(ii,jj))
        fig = plt.imshow(layer_4[jj, ii,:,:])
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

In [ ]:
# torch.save(model.state_dict(), '/media/rich/bigSSD/Net_trainedOnAug_20211025_trainingSet_mouse628_20200903and20200815_simCLR.pth')

In [ ]:
# model = Net()
# model.load_state_dict(torch.load('test_save.pth'))
# model.eval()